In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import numpy as np
# import dynamo as dyn
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.insert(0, "../../scripts/methods/")
from my_paste2 import paste2_align, paste2_align_ref

sys.path.insert(0, "/home/ylu/project")
from utils import *
from tqdm import tqdm

2024-07-30 20:31:37.911756: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-07-30 20:31:38.896302: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-07-30 20:31:38.896392: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-07-30 20:31:38.896399: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

In [2]:
data_folder = "../../data/STARMapPlus_nonrigid_simulation_different_distort/"

method = 'PASTE2'
results_folder = "./nonrigid_different_distort_results/"+method
Path(results_folder).mkdir(parents=True, exist_ok=True)


In [3]:
slice_name = ['A','B','C','D']
tps_slice_name = ['A*','B*','C*','D*']
i = 10
data_name = "well_{}".format(i)
s = ad.read_h5ad('../../data/STARMapPlus/{}.h5ad'.format(data_name))
norm_scale = np.linalg.norm(s.obsm['spatial'].max(0) - s.obsm['spatial'].min(0))
del s
j = 1
slice1 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j) + ".h5ad"))
slice2 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j+1) + ".h5ad"))
repeat = 10

In [ ]:
for l, no in enumerate(np.arange(0,0.22,0.02)):
    if l < 3:
        continue
    alignment_results = []
    for k in range(repeat):
        slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
        slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_{}_r_{}'.format(l, k)].copy()
        align_slices, _, _ = paste2_align_ref(
            models=[slice1, slice2],
            spatial_key="prealign_spatial",
            key_added="align_spatial",
            verbose=True,
            fixed_s=0.99,
            n_sampling=5000,
        )
        
        torch.cuda.empty_cache()
        mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
        mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
        print(f'------iteration: {l}----repeat: {k}-----mae: {mae}---------')
        
        alignment_results.append([mae, mae_var, mse, mse_var, data_name, no, l, k])

    np.save(os.path.join(results_folder, data_name + f"_{method}_differ_distort_{l}.npy"), alignment_results, allow_pickle=True)

/home/ylu/project/Benchmark_alignment_methods/notebooks/STARMapPlus/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/anaconda3/envs/GPSA/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/ylu/anaconda3/envs/GPSA/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/ylu/project/Benchmark_alignment_methods/notebooks/STARMapPlus/../../scripts/methods/my_paste2.py:113: ImplicitModificationWarning: Setting element `.obsm['align_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]
/home/ylu/anaconda3/envs/GPSA/lib/

PASTE2 starts...
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|4.198064e+00|0.000000e+00|0.000000e+00
